In [56]:
!pip install "dask[complete]"
!pip install pyarrow pandas

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries**

In [58]:
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd
import glob
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

In [59]:
pd.set_option('display.max_colwidth', None)

**Read Dataset**

In [60]:

# Directory containing the data files (assuming they are in parquet format)
data_dir = '/content/drive/MyDrive/Data_hasking'

# Get a list of all parquet files
file_list = sorted(glob.glob(f"{data_dir}/*.parquet"))

# Define the date range for filtering (December 3 to December 7, 2023)
start_date = pd.to_datetime('2023-12-03')
end_date = pd.to_datetime('2023-12-07')

# Filter the file list based on the date in the filename
filtered_files = [
    f for f in file_list
    if start_date <= pd.to_datetime(f.split('_')[-1].replace('.parquet', ''), format='%Y%m%d') <= end_date
]



In [61]:
# Load the filtered data into a Dask DataFrame
df = dd.read_parquet(filtered_files)

# Compute the DataFrame to get a Pandas DataFrame
df = df.compute()

In [62]:
df

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,20231203
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,20231203
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,20231203
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,20231203
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,20231203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802043,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794685820,VND,4750.0,4750.0,110000,...,<NA>,N,1|2,98be8d28c01235ba77ff244ca6035dd86086b1ca28ca801a4fca515f57b3b9d6,MOB,HOI SO SHB,A2,VU THI YEN,NaN,20231207
802044,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734517,VND,-3500.0,-3500.0,110000,...,<NA>,N,1|2,4dc1194ff047ca5307f6848f4b69aa562c00e6022e5413205404eb622c803a6a,MOB,HOI SO SHB,FP,<NA>,NaN,20231207
802045,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734516,VND,900.0,900.0,110000,...,<NA>,Y,1|1,983fc2ba457d77883ea2a5160e77b81ffb98dc6e325abec2907a4ed8e43ec90c,NET,HOI SO SHB,FP,<NA>,NaN,20231207
802046,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734518,VND,-850.0,-850.0,110000,...,<NA>,N,1|2,38213e45e4a821d58cc7f7ca3294ba14479168f67bf22ea53ca5a15e2eb8bc60,MOB,HOI SO SHB,FP,<NA>,NaN,20231207


In [63]:
# Filter transactions for amounts exactly equal to 100,000 or -100,000
transactions_amount = df[df['AMOUNT'].isin([100000, -100000])]

In [76]:
# Group by customer and count occurrences of 100,000 and -100,000
transaction_counts = transactions_amount.groupby('CUST_CUSTNO')['AMOUNT'].value_counts().unstack(fill_value=0)

In [65]:
# Select customers with both deposits and withdrawals of 100,000
selected_customers = transaction_counts[(transaction_counts[100000] > 0) & (transaction_counts[-100000] > 0)]

# Create a DataFrame for selected customers and their amounts
selected_customer_df = transactions_amount[transactions_amount['CUST_CUSTNO'].isin(selected_customers.index.tolist())][['CUST_CUSTNO', 'AMOUNT','VALUEDATE']]

# Display the DataFrame
selected_customer_df

,CUST_CUSTNO,AMOUNT,VALUEDATE
7920,76d8b060fe13253fc93ae65072eca87fe32c3a46a34dc9f897b9db42a46d2321,-100000.0,20231203
8304,76d8b060fe13253fc93ae65072eca87fe32c3a46a34dc9f897b9db42a46d2321,100000.0,20231203
10445,fd8c6948039df262db8ff8594af572f11502dbef68419a357199fffa0225a587,-100000.0,20231203
12486,f65f769b3e20dbe269cc8d5ac6c0195fadcd8cddfacc54aa1198f6f8ae0b9b71,100000.0,20231203
12983,d51985753a0011e32a3b62e62a499a5720392d6fce195328fa813b2e320bf34c,100000.0,20231203
...,...,...,...
796323,2fed8edb036a427d8a2b8409e2d4e50f79bbdcd4b6fa6c6addd3b731d161c2a7,-100000.0,20231207
796345,2fed8edb036a427d8a2b8409e2d4e50f79bbdcd4b6fa6c6addd3b731d161c2a7,100000.0,20231207
797375,02005478c6c830b712bd7ca87c82ceb743985f52d3ff4030474da574192a3d1f,100000.0,20231207
797387,02005478c6c830b712bd7ca87c82ceb743985f52d3ff4030474da574192a3d1f,-100000.0,20231207


In [66]:
# Group by customer and date, then count occurrences of each amount
daily_transactions = selected_customer_df.groupby(['CUST_CUSTNO', 'VALUEDATE'])['AMOUNT'].value_counts().unstack(fill_value=0)
daily_transactions

AMOUNT                                                                      -100000.0  \
CUST_CUSTNO                                                      VALUEDATE              
0007c2cfad20bdb52d8285196af079100b824413fdbe41d4f14dedb85c9c5309 20231204           2   
000975b0ee367856502b174994023821120d5730b3fcd384eda36753a7e1e2c7 20231205           1   
                                                                 20231207           1   
000c7a8dc930b1c38d5283351cc9f8ed116b21678639f30ef643cbffb3fe594a 20231205           2   
001921dda5635e50b4f619afc17fbb776adb5556b1ab08e09203048f64843ea9 20231205           1   
...                                                                               ...   
ffb0b16a94c4f6ae1fedfde60bb7d2ec8eeeec35f2c1289bfe65229b6142f52e 20231206           1   
ffbfb9dff789505b49ae502c7d095b631bc5f671dcb89b2d46c78bedb27cb257 20231206           1   
fff19c110a51b1bc27ad3c1522cbca80d8841483dbeb87dc09c5b965e2563950 20231205           1   
fffb93ce64ca2b95eec1756bede5a0b14db89cea65c759868375b5bfc06ee404 20231204           1   
fffc2d43f0d578d0d80242a267269a93328dd82d6d8bc287a6ba713215535dea 20231204           1   

AMOUNT                                                                       100000.0  
CUST_CUSTNO                                                      VALUEDATE             
0007c2cfad20bdb52d8285196af079100b824413fdbe41d4f14dedb85c9c5309 20231204           2  
000975b0ee367856502b174994023821120d5730b3fcd384eda36753a7e1e2c7 20231205           1  
                                                                 20231207           0  
000c7a8dc930b1c38d5283351cc9f8ed116b21678639f30ef643cbffb3fe594a 20231205           1  
001921dda5635e50b4f619afc17fbb776adb5556b1ab08e09203048f64843ea9 20231205           1  
...                                                                               ...  
ffb0b16a94c4f6ae1fedfde60bb7d2ec8eeeec35f2c1289bfe65229b6142f52e 20231206           1  
ffbfb9dff789505b49ae502c7d095b631bc5f671dcb89b2d46c78bedb27cb257 20231206           1  
fff19c110a51b1bc27ad3c1522cbca80d8841483dbeb87dc09c5b965e2563950 20231205           1  
fffb93ce64ca2b95eec1756bede5a0b14db89cea65c759868375b5bfc06ee404 20231204           1  
fffc2d43f0d578d0d80242a267269a93328dd82d6d8bc287a6ba713215535dea 20231204           1  

[4806 rows x 2 columns]

In [67]:
# Filter transactions where both deposit and withdrawal of 100,000 occurred on the same day
transactions_day = daily_transactions[
    (daily_transactions[100000] > 0) & (daily_transactions[-100000] > 0)
]
transactions_day

,AMOUNT,-100000.0,100000.0
CUST_CUSTNO,VALUEDATE,,
0007c2cfad20bdb52d8285196af079100b824413fdbe41d4f14dedb85c9c5309,20231204,2,2
000975b0ee367856502b174994023821120d5730b3fcd384eda36753a7e1e2c7,20231205,1,1
000c7a8dc930b1c38d5283351cc9f8ed116b21678639f30ef643cbffb3fe594a,20231205,2,1
001921dda5635e50b4f619afc17fbb776adb5556b1ab08e09203048f64843ea9,20231205,1,1
0021e74dd8107882257e3587eea5b96b551b1edd3002b3c4d047b6df4c3a3fa2,20231207,1,1
...,...,...,...
ffb0b16a94c4f6ae1fedfde60bb7d2ec8eeeec35f2c1289bfe65229b6142f52e,20231206,1,1
ffbfb9dff789505b49ae502c7d095b631bc5f671dcb89b2d46c78bedb27cb257,20231206,1,1
fff19c110a51b1bc27ad3c1522cbca80d8841483dbeb87dc09c5b965e2563950,20231205,1,1


In [73]:
# Filter transactions of customers
filtered_transactions = transactions_amount[transactions_amount['CUST_CUSTNO'].isin(transactions_day.index.get_level_values(0))]
filtered_transactions


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
7920,76d8b060fe13253fc93ae65072eca87fe32c3a46a34dc9f897b9db42a46d2321,CASA,eb8de8420558cfcf61ffc2e10637467d4e473a5cc4c62492ca04958951d52bb6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391763972,VND,-100000.0,-100000.0,120302,...,<NA>,N,2|1,099a7b52526eb88deaa86718797ca46c46076688dd0083ac9a28dfad9a187d9e,BRC,PGD NINH HOA/CN KHANH HOA,TD,<NA>,NaN,20231203
8304,76d8b060fe13253fc93ae65072eca87fe32c3a46a34dc9f897b9db42a46d2321,CASA,eb8de8420558cfcf61ffc2e10637467d4e473a5cc4c62492ca04958951d52bb6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391885231,VND,100000.0,100000.0,110000,...,<NA>,Y,1|1,0b6ef1f14b7d06708a67208215606a376b7fb1d3f33884bda38fa2b4811b3a5c,NET,HOI SO SHB,FP,<NA>,NaN,20231203
10445,fd8c6948039df262db8ff8594af572f11502dbef68419a357199fffa0225a587,CASA,73df4b36c67a6a2bd26cd8820f4bc8b99ab51dfeb0d6e6d3a8879732661b5a34,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391438742,VND,-100000.0,-100000.0,110000,...,<NA>,N,1|1,7af11b37765dc7c40e3e2528ddbdbcbb80de0b74ecc59fe2adebbb36a06a7187,MOB,HOI SO SHB,A2,NGUYEN HOANG LONG,NaN,20231203
12486,f65f769b3e20dbe269cc8d5ac6c0195fadcd8cddfacc54aa1198f6f8ae0b9b71,CASA,ad03b4b83a1b5261323db00a5c908913e530eef5e05c630a7e88c56f8af148ba,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391933335,VND,100000.0,100000.0,110000,...,X),Y,1|1,a69fa7d1e0e61c228df3803dda9acd1129dfffefd32c7986596d99dee8afcc99,NET,HOI SO SHB,FP,<NA>,NaN,20231203
12983,d51985753a0011e32a3b62e62a499a5720392d6fce195328fa813b2e320bf34c,CASA,2025ceafcab91cff08ceb060802780497de4e5285aa3873dc5b61b48b2be970a,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391949228,VND,100000.0,100000.0,110000,...,<NA>,Y,1|1,86f9acd8693f9fcca9d12e255a71c11c68a630f8ade6000a98c618e35db6c0c9,NET,HOI SO SHB,FP,<NA>,NaN,20231203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796323,2fed8edb036a427d8a2b8409e2d4e50f79bbdcd4b6fa6c6addd3b731d161c2a7,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120795321724,VND,-100000.0,-100000.0,110316,...,<NA>,N,2|1,76bcba36f1318d2067325fe61952707bfc16871b022b000bc7582af96281b8cd,BRC,PGD MAO KHE/CN QUANG NINH,TD,<NA>,NaN,20231207
796345,2fed8edb036a427d8a2b8409e2d4e50f79bbdcd4b6fa6c6addd3b731d161c2a7,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120795321723,VND,100000.0,100000.0,110000,...,<NA>,Y,1|1,5f870cb6315eae836f9c8113c1b88486d9547eb90d581dadeb58fe8734604adb,NET,HOI SO SHB,FP,<NA>,NaN,20231207
797375,02005478c6c830b712bd7ca87c82ceb743985f52d3ff4030474da574192a3d1f,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120795307773,VND,100000.0,100000.0,110000,...,<NA>,Y,1|1,3a351a0229b29fb070207afd7c4dd7756bdf68c148e9610d4e861e692d41ce89,NET,HOI SO SHB,FP,<NA>,NaN,20231207
797387,02005478c6c830b712bd7ca87c82ceb743985f52d3ff4030474da574192a3d1f,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120795307774,VND,-100000.0,-100000.0,110000,...,<NA>,N,1|2,81a9de5e410525071110b8113bc2f473b89e0c8416ea7fdb160ca9325f3d0f72,MOB,HOI SO SHB,FP,<NA>,NaN,20231207
